In [ ]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split

In [ ]:
BASE_PATH_SOA = ''

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(256, kernel_size=3, strides=1, input_shape=(2230, 1), activation=None),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Conv1D(256, kernel_size=3, strides=1, activation=None),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Conv1D(256, kernel_size=3, strides=1, activation=None),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Conv1D(256, kernel_size=3, strides=1, activation=None),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
def read_and_prepare_data(path):
    df = pd.read_csv(path, sep=';', encoding='utf-8')

    y = df['label']
    X = df.drop(columns=['label'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=161194)
    
    train = X_train.astype('float32').to_numpy()
    train = train.reshape(train.shape[0], train.shape[1], 1)
    train_y = y_train.to_numpy()
    
    test = X_test.astype('float32').to_numpy()
    test = test.reshape(test.shape[0], test.shape[1], 1)
    test_y = y_test.to_numpy()
    
    return train, train_y, test, test_y

In [ ]:
class CustomStopper(tf.keras.callbacks.EarlyStopping):
    def __init__(self, monitor='val_loss',
             min_delta=0, patience=0, verbose=0, mode='auto', start_epoch = 15): # add argument for starting epoch
        super(CustomStopper, self).__init__()
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)

In [ ]:
def train(path, name, epochs=60):
    X_train, y_train, X_test, y_test = read_and_prepare_data(path)
    
    logdir = os.path.join(BASE_PATH_SOA, "logs", name + datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    model = build_model()
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    early_stopping_callback = CustomStopper(monitor='val_loss', patience=5)
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tensorboard_callback, early_stopping_callback]
    )
    
    model.save(logdir + '/model')
    
    pd.DataFrame(history.history).to_csv(logdir + '/history_train.csv', sep=';', encoding='utf-8')

    history_eval = model.evaluate(X_test, y_test)

    pd.DataFrame(history_eval).to_csv(logdir + '/history_eval.csv', sep=';', encoding='utf-8')
    
    return model, history

In [ ]:
model, history = train(BASE_PATH_SOA + '64_QF95-SOA.csv', '64_QF95-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '64_QF98-SOA.csv', '64_QF98-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '64_QF100-SOA.csv', '64_QF100-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '128_QF95-SOA.csv', '128_QF95-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '128_QF98-SOA.csv', '128_QF98-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '128_QF100-SOA.csv', '128_QF100-SOA_')

In [ ]:
model, history = train(BASE_PATH_SOA + '64_asc-SOA.csv', '64_ASC-SOA_', 15)

In [ ]:
model, history = train(BASE_PATH_SOA + '64_desc-SOA.csv', '64_DESC-SOA_', 15)

In [ ]:
model, history = train(BASE_PATH_SOA + '128_asc-SOA.csv', '128_ASC-SOA_', 15)

In [ ]:
model, history = train(BASE_PATH_SOA + '128_desc-SOA.csv', '128_DESC-SOA_', 15)